In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# =====================================================
# STEP 1 — LOAD DATA
# =====================================================

train_path = "/content/drive/MyDrive/ML_Project/MultinomialClassification/Dataset/train.csv"
test_path  = "/content/drive/MyDrive/ML_Project/MultinomialClassification/Dataset/test.csv"



df_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

TARGET = "spend_category"
ID_COL = "trip_id"

print("Original Training Shape:", df_raw.shape)
print("Original Test Shape:", test_raw.shape)


# =====================================================
# HELPER — Convert Range (15-30, 90+) to numeric
# =====================================================
def range_to_mid(x):
    x = str(x).strip()
    if x.lower() in ["none", "", "nan", "null"]:
        return np.nan
    if "+" in x:
        return float(x.replace("+", ""))
    if "-" in x:
        a, b = x.split("-")
        return (float(a) + float(b)) / 2
    try:
        return float(x)
    except:
        return np.nan


# =====================================================
# STEP 2 — GLOBAL COLUMN DEFINITIONS
# =====================================================
binary_cols = [
    "is_first_visit","intl_transport_included","accomodation_included",
    "food_included","domestic_transport_included","sightseeing_included",
    "guide_included","insurance_included"
]

categorical_cols = [
    "country","age_group","travel_companions","main_activity",
    "visit_purpose","tour_type","info_source","arrival_weather"
]

numeric_count_cols = ["num_females","num_males","mainland_stay_nights","island_stay_nights"]


# =====================================================
# STEP 3 — REMOVE NULL TARGETS FIRST
# =====================================================
removed_target_nulls = df_raw[TARGET].isnull().sum()
print("Rows removed due to null spend_category:", removed_target_nulls)

df_raw = df_raw[df_raw[TARGET].notnull()].reset_index(drop=True)
print("Training shape after removing null targets:", df_raw.shape)


# =====================================================
# STEP 4 — MAIN PREPROCESSING FUNCTION
# =====================================================
def preprocess_raw_df(df):
    df = df.copy()

    # Clean strings
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].astype(str).str.strip().str.rstrip(',')

    # Binary processing
    for c in binary_cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.lower()
            df[c] = df[c].replace({
                "yes": 1,
                "no": 0,
                "nan": np.nan,
                "none": np.nan,
                "null": np.nan,
                "": np.nan
            })
            df[c] = df[c].fillna(0).astype(int)

    # Numeric count fields
    for c in numeric_count_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    # =====================================================
    # SAFE ORDINAL ENCODING FOR RANGE COLUMNS
    # =====================================================

    # Clean weird string values
    def clean_str(x):
        x = str(x).strip().lower()
        if x in ["nan", "none", "null", ""]:
            return np.nan
        return x

    # Clean raw string columns
    if "days_booked_before_trip" in df.columns:
        df["days_booked_before_trip_clean"] = df["days_booked_before_trip"].apply(clean_str)

    if "total_trip_days" in df.columns:
        df["total_trip_days_clean"] = df["total_trip_days"].apply(clean_str)

    # Define ordinal mappings
    ordinal_days_booked = {
        "1-7": 1,
        "8-14": 2,
        "15-30": 3,
        "31-60": 4,
        "61-90": 5,
        "90+": 6
    }

    ordinal_total_trip = {
        "1-6": 1,
        "7-14": 2,
        "15-30": 3,
        "30+": 4
    }

    # Map → Fill Missing → Convert to int
    if "days_booked_before_trip_clean" in df.columns:
        df["days_booked_before_trip_ord"] = (
            df["days_booked_before_trip_clean"]
                .map(ordinal_days_booked)
        )

        # Fill NaN with mode **of ordinal values**
        df["days_booked_before_trip_ord"].fillna(
            df["days_booked_before_trip_ord"].mode()[0],
            inplace=True
        )

        df["days_booked_before_trip_ord"] = df["days_booked_before_trip_ord"].astype(int)

    if "total_trip_days_clean" in df.columns:
        df["total_trip_days_ord"] = (
            df["total_trip_days_clean"]
                .map(ordinal_total_trip)
        )

        # Fill NaN with mode of ordinal values
        df["total_trip_days_ord"].fillna(
            df["total_trip_days_ord"].mode()[0],
            inplace=True
        )

        df["total_trip_days_ord"] = df["total_trip_days_ord"].astype(int)



    # Special requirements → binary
    if "has_special_requirements" in df.columns:
        df["has_special_req_bin"] = df["has_special_requirements"].astype(str).apply(
            lambda x: 0 if x.lower() in ["none", "", "nan"] else 1
        )

    return df


# =====================================================
# APPLY PREPROCESSING TO TRAIN & TEST
# =====================================================
df = preprocess_raw_df(df_raw).reset_index(drop=True)
test_df = preprocess_raw_df(test_raw).reset_index(drop=True)

print("\nAfter Base Preprocessing:")
print(df.shape)


# =====================================================
# STEP 5 — IMPUTATIONS (NO NaNs must remain)
# =====================================================

# Categorical mode fill
for c in categorical_cols:
    if c in df.columns:
        mode = df[c].mode()[0]
        df[c] = df[c].fillna(mode)
        test_df[c] = test_df[c].fillna(mode)

# =========================================================
# OUTLIER REMOVAL (TRAIN ONLY)
# And count number of rows removed per condition
# =========================================================

clean_df = df.copy()
initial_rows = len(clean_df)

outlier_info = {}

# ----- num_females ≤ 14 -----
before = len(clean_df)
clean_df = clean_df[clean_df["num_females"] <= 10]
after = len(clean_df)
outlier_info["num_females"] = before - after

# ----- Rule 1: num_males ≤ 13 -----
before = len(clean_df)
clean_df = clean_df[clean_df["num_males"] <= 10]
after_rule1 = len(clean_df)
removed_rule1 = before - after_rule1
before_rule2 = len(clean_df)

# Save results
outlier_info["num_males_threshold"] = removed_rule1
print("Removed (num_males > 20):", removed_rule1)


# ----- mainland_stay_nights ≤ 100 -----
before = len(clean_df)
clean_df = clean_df[clean_df["mainland_stay_nights"] <= 90]
after = len(clean_df)
outlier_info["mainland_stay_nights"] = before - after

# ----- island_stay_nights ≤ 21 -----
before = len(clean_df)
clean_df = clean_df[clean_df["island_stay_nights"] <= 60]
after = len(clean_df)
outlier_info["island_stay_nights"] = before - after

final_rows = len(clean_df)

# =========================================================
# PRINT OUTLIER REMOVAL SUMMARY
# =========================================================
print("\n========== OUTLIER REMOVAL SUMMARY ==========")
for col, removed in outlier_info.items():
    print(f"{col}: removed {removed} rows")

print("---------------------------------------------")
print(f"Total rows removed: {initial_rows - final_rows}")
print(f"Final Training Shape after Outlier Removal: {clean_df.shape}")
print("Test Shape (unchanged):", test_df.shape)


# =====================================================
# STEP 5 — FINAL FEATURE DEFINITIONS (No NaNs left)
# =====================================================

numeric_features = [
    "num_females",
    "num_males",
    "mainland_stay_nights",
    "island_stay_nights",
    "days_booked_before_trip_ord",
    "total_trip_days_ord"
]

binary_features = binary_cols + ["has_special_req_bin"]

categorical_features = categorical_cols

all_features = numeric_features + binary_features + categorical_features

clean_df = clean_df[all_features + [TARGET]]
test_df_final = test_df[all_features]




Original Training Shape: (12654, 25)
Original Test Shape: (5852, 24)
Rows removed due to null spend_category: 34
Training shape after removing null targets: (12620, 25)


/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] =


After Base Preprocessing:
(12620, 30)
Removed (num_males > 20): 8

========== OUTLIER REMOVAL SUMMARY ==========
num_females: removed 28 rows
num_males_threshold: removed 8 rows
mainland_stay_nights: removed 27 rows
island_stay_nights: removed 8 rows
---------------------------------------------
Total rows removed: 71
Final Training Shape after Outlier Removal: (12549, 30)
Test Shape (unchanged): (5852, 29)


In [2]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# =====================================================
# STEP X — FEATURE ENGINEERING: K-Means + GMM CLUSTERS
# =====================================================

print("\n=== Running Clustering Feature Engineering (KMeans) ===")

# ----------------------------
# 1. Prepare numeric data only
# ----------------------------
num_only = clean_df[numeric_features].copy()
test_num_only = test_df_final[numeric_features].copy()

# Standardize for clustering
scaler = StandardScaler()
num_scaled = scaler.fit_transform(num_only)
test_num_scaled = scaler.transform(test_num_only)

# ----------------------------
# 2. K-Means Clustering
# ----------------------------
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
clean_df["kmeans_cluster"] = kmeans.fit_predict(num_scaled)
test_df_final["kmeans_cluster"] = kmeans.predict(test_num_scaled)

# # ----------------------------
# # 3. Gaussian Mixture Model (GMM)
# # ----------------------------
# gmm = GaussianMixture(n_components=5, random_state=42)
# clean_df["gmm_cluster"] = gmm.fit_predict(num_scaled)
# test_df_final["gmm_cluster"] = gmm.predict(test_num_scaled)

print("Clustering features added: kmeans_cluster")

# Add to categorical features
categorical_features += ["kmeans_cluster"]

# Update full feature list
all_features = numeric_features + binary_features + categorical_features

# Important: keep only final columns
clean_df = clean_df[all_features + [TARGET]]
test_df_final = test_df_final[all_features]

print("\nFinal shape after clustering:", clean_df.shape)



=== Running Clustering Feature Engineering (KMeans) ===
Clustering features added: kmeans_cluster

Final shape after clustering: (12549, 25)


/tmp/ipython-input-1854590008.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_final["kmeans_cluster"] = kmeans.predict(test_num_scaled)


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocess = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
    ("bin", "passthrough", binary_features)
])


In [4]:
import numpy as np
import os
import optuna
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# =====================================================
# full_df MUST be defined before splitting
# =====================================================

full_df = clean_df.copy()      # ← ADD THIS LINE

# full_df must already contain all_features + TARGET
X = full_df.drop(columns=[TARGET])
y = full_df[TARGET]

# =====================================================
# SPLIT A: 80% train, 10% val, 10% test
# =====================================================
train_80, temp_20, y80, ytemp_20 = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

val_10, test_10, yv10, yt10 = train_test_split(
    temp_20, ytemp_20,
    test_size=0.50,
    stratify=ytemp_20,
    random_state=42
)



# =====================================================
# SIMPLE NEURAL NETWORK + BAYESIAN OPTIMISATION
# =====================================================

def objective(trial):

    # Hyperparameters to tune
    hidden_units = trial.suggest_categorical("hidden_units", [32, 64, 128])
    dropout_rate = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    epochs = trial.suggest_int("epochs", 10, 40)

    input_dim = X80_trans.shape[1]
    num_classes = len(np.unique(y80))

    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(hidden_units, activation=activation),
        layers.Dropout(dropout_rate),
        layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X80_trans, y80,
        validation_data=(Xv10_trans, yv10),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    val_pred = np.argmax(model.predict(Xv10_trans), axis=1)
    val_acc = accuracy_score(yv10, val_pred)

    return val_acc



# =====================================================
# TRAINING FUNCTION
# =====================================================

def train_mlp_bayesian_only(
        X80_df, y80,
        Xv10_df, yv10,
        Xt10_df, yt10):

    # ----------------------------------------
    # Apply your existing ColumnTransformer
    # ----------------------------------------
    preprocess.fit(X80_df)

    global X80_trans, Xv10_trans, Xt10_trans
    X80_trans = preprocess.transform(X80_df)
    Xv10_trans = preprocess.transform(Xv10_df)
    Xt10_trans = preprocess.transform(Xt10_df)

    input_dim = X80_trans.shape[1]
    num_classes = len(np.unique(y80))

    # ----------------------------------------
    # Bayesian Tuning
    # ----------------------------------------
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_params
    print("\nBest Params:", best_params)

    # ----------------------------------------
    # Train final model using best params
    # ----------------------------------------
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(best_params["hidden_units"],
                     activation=best_params["activation"]),
        layers.Dropout(best_params["dropout"]),
        layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(best_params["lr"]),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X80_trans, y80,
        validation_data=(Xv10_trans, yv10),
        epochs=best_params["epochs"],
        batch_size=best_params["batch_size"],
        verbose=1
    )

    # ----------------------------------------
    # Evaluate
    # ----------------------------------------
    val_pred = np.argmax(model.predict(Xv10_trans), axis=1)
    val_acc = accuracy_score(yv10, val_pred)

    test_pred = np.argmax(model.predict(Xt10_trans), axis=1)
    test_acc = accuracy_score(yt10, test_pred)

    # ----------------------------------------
    # SAVE RESULTS EXACTLY LIKE YOUR OLD WAY
    # ----------------------------------------
    BASE_DIR = "/content/drive/MyDrive/ML_Project/MultinomialClassification/NN/SMOTE_BAYES"
    os.makedirs(BASE_DIR, exist_ok=True)

    with open(f"{BASE_DIR}/results.txt", "w") as f:
        f.write(f"Best params: {best_params}\n")
        f.write(f"Validation Accuracy: {val_acc}\n")
        f.write(f"Test Accuracy: {test_acc}\n")

    print("\nDONE — Results saved in:", BASE_DIR)

    # =====================================================
    # FINAL KAGGLE SUBMISSION
    # =====================================================

    # Directory to save output
    out_dir = "/content/drive/MyDrive/ML_Project/MultinomialClassification/NN/SMOTE_BAYES"
    os.makedirs(out_dir, exist_ok=True)

    # Names for file
    model_name = "BayesNN"
    run_name = "80_10_10"

    # Transform test set
    X_final = preprocess.transform(test_df_final[all_features])

    # Predict
    final_probs = model.predict(X_final)
    final_pred  = np.argmax(final_probs, axis=1)

    # Create submission DataFrame
    submission = pd.DataFrame({
        ID_COL: test_raw[ID_COL],   # trip_id
        TARGET: final_pred          # spend_category
    })

    # Save CSV
    submission.to_csv(
        os.path.join(out_dir, f"{model_name}_{run_name}_submission.csv"),
        index=False
    )

    print(f"[DONE] Submission saved → {model_name}_{run_name}_submission.csv ✓")




# =====================================================
# RUN FINAL TRAINING
# =====================================================

print("\n=== Running Bayesian Tuned NN on 80-10-10 ===")

train_mlp_bayesian_only(
    train_80, y80,
    val_10,  yv10,
    test_10, yt10
)



=== Running Bayesian Tuned NN on 80-10-10 ===


[I 2025-11-27 12:32:03,694] A new study created in memory with name: no-name-0d19f942-6735-4c0e-a633-95d0bcce9e19
/tmp/ipython-input-2295545055.py:48: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


[I 2025-11-27 12:32:22,216] Trial 0 finished with value: 0.7402390438247012 and parameters: {'hidden_units': 128, 'dropout': 0.42236118438421666, 'lr': 0.001155247939562752, 'batch_size': 128, 'activation': 'tanh', 'epochs': 31}. Best is trial 0 with value: 0.7402390438247012.
/tmp/ipython-input-2295545055.py:48: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


...

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


[I 2025-11-27 12:36:24,012] Trial 18 finished with value: 0.749003984063745 and parameters: {'hidden_units': 32, 'dropout': 0.493637213293866, 'lr': 0.0006713096004030028, 'batch_size': 64, 'activation': 'relu', 'epochs': 10}. Best is trial 9 with value: 0.7569721115537849.
/tmp/ipython-input-2295545055.py:48: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


[I 2025-11-27 12:36:35,944] Trial 19 finished with value: 0.7250996015936255 and parameters: {'hidden_units': 128, 'dropout': 0.10242415407188915, 'lr': 0.0025633344389403624, 'batch_size': 64, 'activation': 'relu', 'epochs': 15}. Best is trial 9 with value: 0.7569721115537849.



Best Params: {'hidden_units': 64, 'dropout': 0.13469994670735055, 'lr': 0.00023688296672325041, 'batch_size': 64, 'activation': 'relu', 'epochs': 15}
Epoch 1/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4772 - loss: 1.0469 - val_accuracy: 0.7155 - val_loss: 0.7263
Epoch 2/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7098 - loss: 0.7200 - val_accuracy: 0.7291 - val_loss: 0.6566
Epoch 3/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7305 - loss: 0.6424 - val_accuracy: 0.7347 - val_loss: 0.6327
Epoch 4/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7251 - loss: 0.6372 - val_accuracy: 0.7434 - val_loss: 0.6222
Epoch 5/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7429 - loss: 0.6108 - val_accuracy: 0.7538 - val_loss: 0.6155
Epoch 6/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7425 - loss: 0.6128 - val_accuracy: 0.7546 - val_loss: 0.6102
Epoch 7/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7505 - loss: 0.